In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [2]:
(X_train,y_train) , (X_test,y_test)=mnist.load_data()

In [3]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))/255
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))/255

#### Ant colony optimization

In [34]:
def chooseNextVertex(graph, pheremones, currPos):
    graph = 1/graph
    denominator = np.dot(graph[currPos], pheremones[currPos])
    numerator = graph[currPos] * pheremones[currPos]   
    probabilities = numerator/denominator
    rouletteWheel = np.cumsum(probabilities)
    rouletteBall = np.random.random()
    nextVertex = np.where(rouletteWheel >= rouletteBall)[0][0]
    return nextVertex

In [35]:
def traverse(graph, pheremones, start, end):
    curr = start
    path = [curr]
    cost = 0
    prev = start 
    while curr != end:
        nextVertex = chooseNextVertex(graph, pheremones, curr)
        while nextVertex == prev:
            nextVertex = chooseNextVertex(graph, pheremones, curr)
        cost += graph[curr][nextVertex]
        path += [nextVertex]
        prev = curr
        curr = nextVertex
    return path, cost

In [36]:
def releaseGeneration(graph, pheremones, start, end, size = 10):
    paths = []
    costs = []
    for i in range(size):
        p, c = traverse(graph, pheremones, start, end)
        paths += [p]
        costs += [c]
    costs = np.array(costs)    
    
    return paths, costs

In [37]:
def updatePheremones(paths, costs, pheremones, decay = 0):
    pheremones = (1-decay)*pheremones
    
    costs = 1/costs
    for p in range(len(paths)):
        path = paths[p]
        for i in range(len(path) - 1):
            pheremones[path[i]][path[i+1]] += costs[p]
    
    return pheremones
            

In [38]:
def generateProblem(size, density):
    graph = np.full((size, size), np.inf)
    
    for i in range(len(graph)):
        for j in range(i, len(graph)):
            if np.random.random() < density:
                if i!=j:
                    w = np.random.randint(1,20)
                    graph[i][j] = w
                    graph[j][i] = w
                    
    return graph

In [4]:
model=Sequential()

In [5]:
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1))) #this is for convolution layer

In [6]:
model.add(MaxPool2D(2,2)) #this is for pooling layer

In [7]:
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
1875/1875 [==============================] - 43s 22ms/step - loss: 0.1599 - accuracy: 0.9529
Epoch 2/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.0555 - accuracy: 0.9829
Epoch 3/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0354 - accuracy: 0.9891
Epoch 4/10
1875/1875 [==============================] - 44s 24ms/step - loss: 0.0239 - accuracy: 0.9921
Epoch 5/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0168 - accuracy: 0.9947
Epoch 6/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0119 - accuracy: 0.9961
Epoch 7/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0090 - accuracy: 0.9972
Epoch 8/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0077 - accuracy: 0.9973
Epoch 9/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0062 - accuracy: 0.9980
Epoch 10/10
1875/1875 [==============================] - 28s 15m

In [12]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.0604 - accuracy: 0.9858


[0.06039034575223923, 0.98580002784729]